**Regresión: Métricas de Regresión**

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

// 1. Crear una sesión de Spark
val spark = SparkSession.builder()
  .appName("Regression Metrics Example")
  .master("local[*]")
  .getOrCreate()

// 2. Cargar el dataset winequality-red
val data = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("winequality-red.csv") // Reemplaza con la ruta correcta

// 3. Preparar los datos: Crear un vector de características
val featureColumns = Array(
  "fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar",
  "chlorides", "free_sulf_dioxide", "total_sulfur_dioxide", "density",
  "pH", "sulphates", "alcohol"
)

val assembler = new VectorAssembler()
  .setInputCols(featureColumns)
  .setOutputCol("features")

val assembledData = assembler.transform(data)

// 4. Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%)
val Array(trainingData, testData) = assembledData.randomSplit(Array(0.8, 0.2), seed = 42)

// 5. Entrenar un modelo de regresión (Random Forest)
val rf = new RandomForestRegressor()
  .setLabelCol("quality")
  .setFeaturesCol("features")
  .setNumTrees(50)
  .setSeed(42)

val model = rf.fit(trainingData)

// 6. Hacer predicciones sobre el conjunto de prueba
val predictions = model.transform(testData)

// 7. Calcular métricas de regresión
val evaluator = new RegressionEvaluator()
  .setLabelCol("quality")
  .setPredictionCol("prediction")

val rmse = evaluator.setMetricName("rmse").evaluate(predictions)
val r2 = evaluator.setMetricName("r2").evaluate(predictions)
val mae = evaluator.setMetricName("mae").evaluate(predictions)

println(s"RMSE: $rmse")
println(s"R²: $r2")
println(s"MAE: $mae")

// 8. Detener la sesión de Spark
spark.stop()

 ### Interpretación de Resultados

- **Regresión**:

    - **RMSE**: El error promedio en la predicción de la calidad del vino.

    - **R²**: Cuánta variabilidad en la calidad del vino es explicada por el modelo.

    - MAE**: El error absoluto promedio en la predicción.